## Code for parameters grid:
Here is reported the code for obtaining the grid of digital biomarkers (in this case we use resting-state PSD and FC quantitites) for various combinations of model biomarkers:

References: 

[1] L. G. Amato, A. A. Vergani, M. Lassi, C. Fabbiani, S. Mazzeo, R. Burali, B. Nacmias, S. Sorbi, R. Mannella, A. Grippo, V. Bessi, A. Mazzoni, Personalized modeling of Alzheimer’s disease progression estimates neurodegeneration severity from EEG recordings. Alzheimers Dement. Diagn. Assess. Dis. Monit. 16, e12526 (2024).

[2] L. G. Amato, A. A. Vergani, M. Lassi, J. Carpaneto, S. Mazzeo, V. Moschini, R. Burali, G. Salvestrini, C. Fabbiani, G. Giacomucci, G. Galdo, C. Morinelli, F. Emiliani, M. Scarpino, S. Padiglioni, B. Nacmias, S. Sorbi, A. Grippo, V. Bessi, A. Mazzoni, Personalized brain models link cognitive decline progression to underlying synaptic and connectivity degeneration. Alzheimers Res. Ther. 17, 74 (2025).

In [ ]:
# Online visualization.
%pylab nbagg

import sys
from mpl_toolkits.mplot3d import Axes3D

# Import tvb library.
import numpy as np
import tvb.simulator.lab as tvb
from tvb.basic.neotraits.api import Final, List
from tvb.simulator.lab import *
from tvb.datatypes import graph
from tvb.datatypes.time_series import TimeSeriesRegion 
from tvb.analyzers.info import sampen
import tvb.simulator.plot.power_spectra_interactive as ps_int

# Import a bunch of stuff to ease command line usage.
from scipy.stats import entropy, chi2_contingency, circvar, pearsonr, ttest_ind
from scipy import signal
from scipy.signal import butter, lfilter, freqz, hilbert, coherence, filtfilt
from scipy.special import xlogy
import matplotlib.pyplot as plt
import numpy as np
import time as tm

In [ ]:
%matplotlib inline



def norma(arr, a=0, b=1):
    """
    Normalizes the input array to the range [a, b].

    Parameters:
    arr (numpy.ndarray): Input array to be normalized.
    a (float): Lower bound of the desired range.
    b (float): Upper bound of the desired range.

    Returns:
    numpy.ndarray: The normalized array with values in the range [a, b].
    """
    arr_min = np.min(arr)
    arr_max = np.max(arr)
    
    # Avoid division by zero if the array has identical elements
    if arr_min == arr_max:
        return np.full_like(arr, a)
    
    # Normalize to [0, 1]
    normalized_arr = (arr - arr_min) / (arr_max - arr_min)
    
    # Scale to [a, b]
    scaled_arr = a + (normalized_arr * (b - a))
    
    return scaled_arr

def mean_channels(mask1, mask2, signal, chlist):
    summ = 0
    for ch in chlist:
        summ += signal[mask1:mask2, 0, ch, 0]
    summ /= len(chlist)
    return summ

def std_channels(mask1, mask2, signal, chlist):
    summ = 0
    mean = mean_channels(mask1, mask2, signal, chlist)
    for ch in chlist:
        summ += (signal[mask1:mask2, 0, ch, 0] - mean)**2
    #summ = sqrt(summ)
    summ /= len(chlist)-1
    return summ


def ev(tsr):
    input_shape = tsr.data.shape
    result_shape = (input_shape[2], input_shape[2], input_shape[1], input_shape[3])
    result = numpy.zeros(result_shape)

    for mode in range(result_shape[3]):
        for var in range(result_shape[2]):
            data = tsr.data[1000:,var,:, mode].squeeze()
            result[:, :, var, mode] = numpy.corrcoef(data.T)

    corr_coeff = graph.CorrelationCoefficients(source=tsr, array_data=result)
    return corr_coeff


def psd_matrix(time_series):
    result = np.zeros((freq, n_regions))
    for x in range (n_regions):
        result[:, x] = signal.welch(time_series[:, x], freq_sample, nperseg = 10000)[1]
    return result


def butter_bandpass(lowcut, highcut, fs, order=1):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return b, a


freq_sample = 1024
lowcut = 0.5
highcut = 45
order = 4

n_seg = 2048
freq_sample = n_seg//2
freq = freq_sample+1 
window = 'hamming'


def filter_bandpass(signal, lowcut, highcut, fs):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    signal_bp = filtfilt(b, a, signal, axis = 0)
    return signal_bp


def compute_psd(time_series): 
    
    signal_bp = np.zeros((len(time_series[:,0,0,0]), len(time_series[0,0,:,0])))


    Welch_signal_bp = np.zeros((freq, freq, len(time_series[0,0,:,0])))
    
    for i in range (len(time_series[0,0,:,0])):
        signal_bp[:,i] = filter_bandpass(time_series[:,0,i,0], lowcut, highcut, freq_sample)
        Welch_signal_bp[:, 0, i], Welch_signal_bp[0, :, i] = signal.welch(signal_bp[:,i],
                                                                                    freq_sample, nperseg=n_seg, 
                                                                                    noverlap=n_seg//2, window=window)
        
    
    welch_mean = np.mean(Welch_signal_bp[0, :, :], axis = 1)
    welch_mean /= np.max(welch_mean) #Normalize
    welch_sem =  np.std(Welch_signal_bp[0, :, :], axis = 1)
    welch_sem /= np.max(welch_mean) #Normalize
    
    return Welch_signal_bp[:, 0, 0], Welch_signal_bp[0, :, :]  #, welch_mean, welch_sem

def reduce(li):
    result = [(x+y)/2.0 for x, y in zip(li[::2], li[1::2])]
    if len(li) % 2:
        result.append(li[-1])
    return result


def binary_FC(time_series, period = 1, th = 0.13):
    
    con = connectivity.Connectivity.from_file('connectivity_76.zip') 
     # Build a TimeSeries Dataype.
    tsr = TimeSeriesRegion(connectivity=con,
                           data=time_series[:,:,:,:],                            #in TVB 4D format
                           sample_period=period) #in ms
    tsr.configure()


    result_tavg = ev(tsr)
    corr_tavg = result_tavg.array_data[..., 0, 0]
    corr_tavg_th = corr_tavg > th


    pmatrix = np.zeros((76,76))

    for x in range(76):
        np.nan_to_num(pmatrix[x,:], copy = False)
        np.nan_to_num(time_series, copy = False)
    for x in range(76):    
        for y in range(76):
            
#             print(pearsonr(time_series[:,0,x,:], time_series[:,0,y,:])[1])
            pmatrix[x][y] = pearsonr(time_series[:,0,x,:], time_series[:,0,y,:])[1]

    pmatrix_th = pmatrix < 0.05
    corr_tavg_th = np.multiply(corr_tavg_th, pmatrix_th)
    mean_corr = corr_tavg_th.mean()
    
    return corr_tavg_th

In [ ]:
#define impaired regions (taken from Amyloid Braak stage I)
impaired_regions = [9, 21, 22, 30, 31, 32, 34, 59, 60, 62, 68, 69, 70, 72]
#######################################################################


'''We determine model parameters as explained in the paper (eqs 3 and 4), including a neuroplasticity parameter np
(which multiplies cp in the second row of equations 3)'''

n_steps = 50 #How many steps you want to use for changing t_e and t_i
a_array = np.linspace(0.100,0.112,n_steps) #This is the array of considered t_e values, TVB uses'a' as a parameter which is 1/t_e
b_array = np.linspace(0.05,0.025,n_steps) #This is the array of considered t_i values, TVB uses 'b' as a parameter which is 1/t_i

your_path = ''
structural_connectivities = np.load(your_path + 'structural_connectivities.npy')

#Define important constants
simul_length = 4000 #How long the simulation is gonna be by default (at least 4000)
def Simulate(cp, lp, np_parameter, g, velocity, noise, regions = impaired_regions, sim_time = 10000):
    '''This function simulates signals at different levels of neurodegeneration parameters. The regions in which hypoinhibition is modelled are passed 
    through the "regions" argument which by default is equal to the "impaired_regions" listed above''' 

    con = connectivity.Connectivity.from_file('connectivity_76.zip') 
    con.weights = structural_connectivities[cp, np_parameter, :, :] #This alters the connectome according to equation 3 for a combination of cp and np
    # cp is comprised between o and 2 with 50 equal steps, while neuroplasticity is uniformally distributed between 0 and 2 with steps of 0.25 amplitude (plus a value of 2.5 at the end)
    con.weights = con.weights / np.max(con.weights)              #normalised by maximum weight  
    con.weights *= g
    con.speed = np.array(velocity)                               #To select conduction velocity of the signal
    con.configure()
    
    a_value = a_array[lp] #this determines the value of t_e
    b_value = b_array[lp] #this determines the value of t_i

    #altering inhibition in impaired regions:
    b_regions = np.ones(76)*0.05 #default value
    b_regions [regions]  = np.ones(len(regions)) * b_value # impairs b value (and therefore t_i) only in selected regions


    # setup model based on paper's parameters
    model_pars = dict(
        A=3.25, #excitatory PSP
        B=22, #inhibitory PSP
        v0=6.0,
        a=a_value, # increased with respect to the healthy case. (also remember that TVB uses ms, not s)
        b=b_regions,    # decreased with respect to the healthy case. (also remember that TVB uses ms, not s)
        r=0.56,
        nu_max=0.0025, # e0 in the JR original paper
        # TVB factors C_i into J*a_i, e.g. C1 = a_1 * J
        J=128,
        a_1=1.0,
        a_2=0.8,
        a_3= 0.25,
        a_4= 0.25,
        mu=0.22, # baseline input, avg of 120 and 320 pulses/s
    )

    # implement JR + afferent PSP for setting the JR in the right portion of the phase space
    class JRPSP(tvb.models.JansenRit):
        state_variable_range = Final(
            default={"y0": np.array([-1.0, 1.0]),
                     "y1": np.array([-500.0, 500.0]),
                     "y2": np.array([-50.0, 50.0]),
                     "y3": np.array([-6.0, 6.0]),
                     "y4": np.array([-20.0, 20.0]),
                     "y5": np.array([-500.0, 500.0]),
                     "y6": np.array([-20.0, 20.0]),
                     "y7": np.array([-500.0, 500.0])})

        variables_of_interest = List(
            of=str,
            label="Variables watched by Monitors",
            choices=("y0", "y1", "y2", "y3", "y4", "y5", "y6", "y7"),
            default=("y0", "y1", "y2", "y3"))

        state_variables = tuple('y0 y1 y2 y3 y4 y5 y6 y7'.split())
        _nvar = 8
        cvar = np.array([6], dtype=np.int32)

        def dfun(self, state_variables, coupling, local_coupling=0.0):
            dy = np.zeros((8, state_variables.shape[1], 1))
            # TVB's JR is eq 6 only
            dy[:6] = super().dfun(state_variables[:6], coupling, local_coupling)
            # tack on PSP for efferent following eq 8
            # NB with this, only y12 is coupling var for TVB
            y0, y1, y2, y3, y4, y5, y6, y7 = state_variables
            a_d = self.a / 3.0
            sigm_y1_y2 = 2.0 * self.nu_max / (1.0 + np.exp(self.r * (self.v0 - (y1 - y2))))
            dy[6] = y7
            dy[7] = self.A * a_d * sigm_y1_y2 - 2.0 * a_d * y7 - self.a**2 * y6
            return dy

    # factor out noise from dy4 = A a (p(t) + ...) as y4 += dt (...) + A a dW_t
    # this allows us to model the noise as TVB does it, though scaling requires experiment
    nsig = np.zeros((8, 76, 1))
    nsig[4] = model_pars['A'] * model_pars['a'] * (.320 - .120) * noise
    noise = tvb.noise.Additive(nsig=nsig)

    #setting up monitors:
    period = 1. #sampling time in terms of simulation time (total number of timepoints = sim_time / period)
    mon_tavg = monitors.TemporalAverage(period=period)
    mon_EEG = monitors.EEG.from_file(sensors_fname='eeg_brainstorm_65.txt', 
                                                                  projection_fname='projection_eeg_65_surface_16k.npy',
                                                                  rm_f_name='regionMapping_16k_76.txt',
                                                                  period=period
                                                                  )
    
    #Bundling
    what_to_watch = (mon_tavg, mon_EEG)
    

    sim = simulator.Simulator(connectivity=con,
                              conduction_speed=np.float(con.speed),
                              model=JRPSP(
                              variables_of_interest=("y0", "y1", "y2", "y3", "y4", "y5", "y6", "y7"),
                              **{k: np.array(v) for k, v in model_pars.items()}),
                              coupling=tvb.coupling.Sigmoidal(a=np.array([10.0])),
                              integrator=tvb.integrators.EulerStochastic(dt=0.1, noise=noise),
                              monitors=what_to_watch,
                              simulation_length=sim_time)
    sim.configure()

    (ttavg, tavg), (teeg, eeg) = sim.run(simulation_length=sim_time) #tavg is the name of signals in region space

    return ttavg, tavg, teeg, eeg # ttavg and teeg are the timestamps


def preprocess(ttavg, tavg, teeg, eeg, PSD = True, normalize = True, cut = 1000):

    
    if PSD:
        #Discarding initial transient for PSD analysis
        ttavg = ttavg[cut:]
        tavg = tavg[cut:, :, :, :]
                           
        teeg = teeg[cut:]
        eeg = eeg[cut:, :, :, :]
        
        ttavg -= cut
        teeg -= cut
        

    if normalize:
        #Normalizing and subtracting average
        tavg /= (np.max(tavg,0) - np.min(tavg,0 ))
        tavg -= np.mean(tavg, 0)
        eeg /= (np.max(eeg,0) - np.min(eeg,0 ))
        eeg -= np.mean(eeg, 0)
                           
    return ttavg, tavg, teeg, eeg


In [ ]:
#First grid search to identify values of structural quantities (these will not be altered by the disease parameters
global_coupling_val = [0.1,1,10]
velocity_val = [1,10,np.inf]
noise_val = [50e-7, 50e-3, 5]

n_regions = len(connectivity.Connectivity.from_file('connectivity_76.zip').weights) #Standard TVB connectome
n_channels = 65

tavgs = np.zeros((len(global_coupling_val), len(velocity_val), len(noise_val), simul_length, n_regions))
eegs = np.zeros((len(global_coupling_val), len(velocity_val), len(noise_val), simul_length, n_channels))

for i in range(len(global_coupling_val)):
    for j in range(len(velocity_val)):
        for k in range(len(noise_val)):
            ttavg, tavg, teeg, eeg = Simulate(cp = 0, lp = 0, np_parameter = 0, g=global_coupling_val[i],
                                                              velocity=velocity_val[j], noise=noise_val[k], sim_time = simul_length)
#             ttavg, tavg_healthy, teeg, eeg_healthy = preprocess(ttavg, tavg_healthy, teeg, eeg_healthy) #if you want to preprocess
                                                                                                          
            tavgs[i,j,k,:,:] = tavg[:, 0, :, 0]
            eegs[i,j,k,:,:] = eeg[:, 0, :, 0]
            
#Here insert your check to identify best combination (or simply visually check as in "AD_model_simulations" and code below)


In [ ]:
# For visual checking the best solution in terms of global coupling, velocity, noise
# dimensions of eegs are [global coupling, velocity, noise, time, channels]

# eeg = eegs[0,2,2, 1000:, :]

# eeg /= (np.max(eeg,0) - np.min(eeg,0 )) #normalization step
# eeg -= np.mean(eeg, 0)  #re-referencing step

# teeg = np.linspace(0,np.shape(eeg)[0], np.shape(eeg)[0]) #creating an array of time-steps


# plt.figure(figsize=(10,12))
# plt.plot(teeg, eeg + np.r_[:len(mon_EEG.sensors.labels)]) # the np.r_[:76] is addedd to separate channels or regions
# plt.yticks(np.r_[:len(mon_EEG.sensors.labels)], mon_EEG.sensors.labels, fontsize = 12)
# plt.title("Brain activity (EEG) ", fontsize = 18)
# plt.xlabel('time (ms)', fontsize = 16)
# plt.show()

In [ ]:
# Second grid search to identify digital biomarkers
psd_array = np.zeros((n_steps,n_steps,n_steps,91, n_regions))
FC_array = np.zeros((n_steps,n_steps,n_steps,n_regions,n_regions))

mean_FCs = np.zeros((n_steps, n_steps, n_steps))
mean_PSD_alphas = np.zeros((n_steps, n_steps, n_steps))

#Here we simulate signals and compute digital biomarkers from virtual patients

###########################################################
for i in range(n_steps):
    for j in range(50):      # the SC matrices here uploaded consider 50 values for cp and 10 for np, if you need more use equations 2 and 3
        for k in range(10):  # from the paper (DOI: https://www.doi.org/10.1186/s13195-025-01718-6) considering additional values
        
            # Suppose that at the previous step the optimal parameter combination was global_coupling_val[0], velocity_val[0], noise=noise_val[0]
            ttavg, tavg, teeg, eeg = Simulate(cp = i, lp = j, np_parameter = k, g=global_coupling_val[0],
                                             velocity=velocity_val[0], noise=noise_val[0], sim_time = simul_length)

            ttavg, tavg, teeg, eeg = preprocess(ttavg, tavg, teeg, eeg)
            print(np.shape(tavg))


            freqs_tavg, Welch_tavg = compute_psd(tavg)

            psd_array[i,j,k,:,:] = Welch_tavg[:91, :] #This stores the PSD
            FC_array[i,j,k,:,:] = binary_FC(tavg) #This stores the FC

            mean_PSD_alphas[i,j,k] = np.sum(psd_array[i,j,k, 16:26, :].mean(axis=1)) / np.sum(psd_array[i,j,k, :, :].mean(axis=1)) #Compute relative alpha power
            mean_FCs[i,j,k] = psd_array[i,j,k, :, :].mean() #Compute average FC

            print(i*n_steps**2 + j*n_steps + k) #Checks the current step